# Preparing data

In [1]:
stoplist = set('for a of the and to rt'.split())

In [2]:
import pickle

## A small corpus

In [3]:
corpus = [
    'Human machine interface for Lab ABC computer applications',
    'A survey of user opinion of computer system response time',
    'The EPS user interface management system',
    'System and human system engineering testing of EPS',
    'Relation of user-perceived response time to error measurement',
    'The generation of random, binary, unordered trees',
    'The intersection graph of paths in trees',
    'Graph minors IV: Width of trees and well-quasi-ordering',
    'Graph minors: A survey'
]

In [4]:
with open('deerwester.txt', 'wb') as f:
    pickle.dump(corpus, f)

In [5]:
# creating generator object for streaming tweets
class Tweets:
    def __iter__(self):
        for tweet in pickle.load(open('deerwester.txt', 'rb')):
            yield tweet

In [6]:
# streaming corpus and storing documents in bow representation
import re
from collections import defaultdict

tweets = Tweets()
token2id = {}
# token2id : dict of (token(str), tokenId(int))
idf = defaultdict(int)
# idf: dict of (tokenId, frequency of tokenId in corpus)
docs2bow = []
# docs2bow: list of [doc2bow]
# doc2bow: list of (tokenIds in doc, frequency of tokenId in doc)
for docno, tweet in enumerate(tweets):
    # lowering tweets and removing punctuations from it, then splitting
    document = re.sub(r'[-,:;|.!?*()+&/~<>="]', ' ', tweet.lower()).split()
    counter = defaultdict(int)
    # counter: dict of (tokenIds in doc, frequency of tokenId in doc)
    for word in document:
        if word in stoplist: continue   # check word by stoplist
        if word not in token2id: token2id[word] = len(token2id) # add word as a token if seen for the first time
        counter[word] += 1
        idf[token2id[word]] += 1
    # creating doc2bow for this doc
    doc2bow = [(token2id[word], freq) for word, freq in counter.items()]
    print(docno, doc2bow)
    # append doc2bow to docs2bow
    docs2bow.append(doc2bow)

0 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
1 [(7, 1), (8, 1), (9, 1), (5, 1), (10, 1), (11, 1), (12, 1)]
2 [(13, 1), (8, 1), (2, 1), (14, 1), (10, 1)]
3 [(10, 2), (0, 1), (15, 1), (16, 1), (13, 1)]
4 [(17, 1), (8, 1), (18, 1), (11, 1), (12, 1), (19, 1), (20, 1)]
5 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]
6 [(26, 1), (27, 1), (28, 1), (29, 1), (25, 1)]
7 [(27, 1), (30, 1), (31, 1), (32, 1), (25, 1), (33, 1), (34, 1), (35, 1)]
8 [(27, 1), (30, 1), (7, 1)]


## A user timeline corpus

In [116]:
import tweepy as tw

# load keys
with open('keys.txt', 'rb') as f:
    keys = pickle.load(f)
# define keys
consumer_key = keys['consumer_key']
consumer_secret = keys['consumer_secret']
access_token = keys['access_token']
access_token_secret = keys['access_token_secret']
# authenticate and create api object
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [117]:
# generator object for iterating through user timeline
class Tweets():
    def __init__(self, pagination_num=3):
        self.pagination_num = pagination_num
        self.cursor = tw.Cursor(api.user_timeline, id="unicef", # id = "indykaila"
                              exclude_replies=True,
                              include_rts=True,
                              tweet_mode='extended').pages(self.pagination_num)
    def __iter__(self):
        for page in self.cursor:
            for status in page:
                yield status.full_text

In [129]:
# streaming corpus and storing documents in bow representation
import re
from collections import defaultdict
from IPython.display import clear_output

tweets = Tweets(100)
token2id = {}
idf = defaultdict(int)
docs2bow = []
for docno, tweet in enumerate(tweets):
    # remove links from tweets
    tweet = re.sub(r'\bhttps:\S+', '', tweet.lower())
    # print(tweet)
    document = re.sub(r'[-,:;|.!?*()+&/~<>="]', ' ', tweet).split()
    # print(document)
    counter = defaultdict(int)
    for word in document:
        if word in stoplist: continue
        if word not in token2id: token2id[word] = len(token2id)
        counter[word] += 1
        idf[token2id[word]] += 1
    doc2bow = [(token2id[word], freq) for word, freq in counter.items()]
    # print(docno, doc2bow)
    print(docno)
    docs2bow.append(doc2bow)
clear_output()
print(docno)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

TweepError: Failed to send request: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Corpus read

In [130]:
len(token2id)

3494

In [8]:
token2id

{'human': 0,
 'machine': 1,
 'interface': 2,
 'lab': 3,
 'abc': 4,
 'computer': 5,
 'applications': 6,
 'survey': 7,
 'user': 8,
 'opinion': 9,
 'system': 10,
 'response': 11,
 'time': 12,
 'eps': 13,
 'management': 14,
 'engineering': 15,
 'testing': 16,
 'relation': 17,
 'perceived': 18,
 'error': 19,
 'measurement': 20,
 'generation': 21,
 'random': 22,
 'binary': 23,
 'unordered': 24,
 'trees': 25,
 'intersection': 26,
 'graph': 27,
 'paths': 28,
 'in': 29,
 'minors': 30,
 'iv': 31,
 'width': 32,
 'well': 33,
 'quasi': 34,
 'ordering': 35}

In [131]:
len(idf)

3494

In [10]:
idf

defaultdict(int,
            {0: 2,
             1: 1,
             2: 2,
             3: 1,
             4: 1,
             5: 2,
             6: 1,
             7: 2,
             8: 3,
             9: 1,
             10: 4,
             11: 2,
             12: 2,
             13: 2,
             14: 1,
             15: 1,
             16: 1,
             17: 1,
             18: 1,
             19: 1,
             20: 1,
             21: 1,
             22: 1,
             23: 1,
             24: 1,
             25: 3,
             26: 1,
             27: 3,
             28: 1,
             29: 1,
             30: 2,
             31: 1,
             32: 1,
             33: 1,
             34: 1,
             35: 1})

In [132]:
len(docs2bow)

772

In [12]:
docs2bow

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (5, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (8, 1), (2, 1), (14, 1), (10, 1)],
 [(10, 2), (0, 1), (15, 1), (16, 1), (13, 1)],
 [(17, 1), (8, 1), (18, 1), (11, 1), (12, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(26, 1), (27, 1), (28, 1), (29, 1), (25, 1)],
 [(27, 1), (30, 1), (31, 1), (32, 1), (25, 1), (33, 1), (34, 1), (35, 1)],
 [(27, 1), (30, 1), (7, 1)]]

## Filter once words

In [133]:
# filter once words
bad_ids = set(tokenid for tokenid, freq in idf.items() if freq == 1)

In [134]:
len(bad_ids)

1829

In [15]:
bad_ids

{1,
 3,
 4,
 6,
 9,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 26,
 28,
 29,
 31,
 32,
 33,
 34,
 35}

In [135]:
# update token2id and idf which filtered once words
token2id = {token: tokenid for token, tokenid in token2id.items() if idf[tokenid] > 1}
idf = {tokenid: freq for tokenid, freq in idf.items() if freq > 1}

In [136]:
len(token2id)

1665

In [18]:
token2id

{'human': 0,
 'interface': 2,
 'computer': 5,
 'survey': 7,
 'user': 8,
 'system': 10,
 'response': 11,
 'time': 12,
 'eps': 13,
 'trees': 25,
 'graph': 27,
 'minors': 30}

In [19]:
idf

{0: 2, 2: 2, 5: 2, 7: 2, 8: 3, 10: 4, 11: 2, 12: 2, 13: 2, 25: 3, 27: 3, 30: 2}

In [137]:
# idmap: maps old tokenIds to new ordered tokenIds
idmap = dict(zip(sorted(token2id.values()), range(len(token2id))))

In [138]:
len(idmap)

1665

In [22]:
idmap

{0: 0,
 2: 1,
 5: 2,
 7: 3,
 8: 4,
 10: 5,
 11: 6,
 12: 7,
 13: 8,
 25: 9,
 27: 10,
 30: 11}

In [139]:
# note this cell is one time run
# compactify token2id and idf
token2id = {token: idmap[tokenid] for token, tokenid in token2id.items()}
idf = {idmap[tokenid]: freq for tokenid, freq in idf.items()}

In [24]:
token2id

{'human': 0,
 'interface': 1,
 'computer': 2,
 'survey': 3,
 'user': 4,
 'system': 5,
 'response': 6,
 'time': 7,
 'eps': 8,
 'trees': 9,
 'graph': 10,
 'minors': 11}

In [25]:
idf

{0: 2, 1: 2, 2: 2, 3: 2, 4: 3, 5: 4, 6: 2, 7: 2, 8: 2, 9: 3, 10: 3, 11: 2}

In [26]:
# token2id changed, but docs2bow still has the same old tokenIds
docs2bow

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (5, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (8, 1), (2, 1), (14, 1), (10, 1)],
 [(10, 2), (0, 1), (15, 1), (16, 1), (13, 1)],
 [(17, 1), (8, 1), (18, 1), (11, 1), (12, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(26, 1), (27, 1), (28, 1), (29, 1), (25, 1)],
 [(27, 1), (30, 1), (31, 1), (32, 1), (25, 1), (33, 1), (34, 1), (35, 1)],
 [(27, 1), (30, 1), (7, 1)]]

In [140]:
# rebuild docs2bow based on new token2id
docs2bow = [
    [(idmap[tokenid], docfreq) for tokenid, docfreq in doc2bow if tokenid not in bad_ids]
    for doc2bow in docs2bow
]

## `docs2bow` ready to use

In [28]:
docs2bow

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (2, 1), (5, 1), (6, 1), (7, 1)],
 [(8, 1), (4, 1), (1, 1), (5, 1)],
 [(5, 2), (0, 1), (8, 1)],
 [(4, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(10, 1), (9, 1)],
 [(10, 1), (11, 1), (9, 1)],
 [(10, 1), (11, 1), (3, 1)]]

In [141]:
# save the corpus in bow representation
with open('corpus2bow.txt', 'wb') as f:
    pickle.dump(docs2bow, f)

## Additional

In [142]:
# not necessary, just n.shape needed for further uses
# note that we don't use n(d,w) matrix in computations, n(d,w) is presented in docs2bow as ndw that you'll see
# creating words2bod by docs2bow -> n -> n.T -> words2bod
# words2bod shows each word appeared in which docs
import numpy as np
# n: numpy array of n[d][w] = n(d,w)
# d = document number, w = word's tokenId
n = np.zeros((len(docs2bow), len(idf)))
for docno, doc2bow in enumerate(docs2bow):
    for tokenid, docfreq in doc2bow:
        n[docno, tokenid] += docfreq
n

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [143]:
n.shape

(772, 1665)

In [32]:
n.T

array([[1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 1., 2., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1.]])

In [33]:
words2bod = [
    [(tokenid, int(docfreq)) for tokenid, docfreq in enumerate(rows) if docfreq != 0]
    for rows in n.T
]

In [34]:
words2bod

[[(0, 1), (3, 1)],
 [(0, 1), (2, 1)],
 [(0, 1), (1, 1)],
 [(1, 1), (8, 1)],
 [(1, 1), (2, 1), (4, 1)],
 [(1, 1), (2, 1), (3, 2)],
 [(1, 1), (4, 1)],
 [(1, 1), (4, 1)],
 [(2, 1), (3, 1)],
 [(5, 1), (6, 1), (7, 1)],
 [(6, 1), (7, 1), (8, 1)],
 [(7, 1), (8, 1)]]

# PLSI model

## Parameters and likelihood

In [144]:
K = 5   # K: number of topics considered, namely the size of latent semantic set Z

In [145]:
from numpy.random import rand

def random_init_pars(K, nshape):
    N, M = nshape   # N = number of documents, M = number of tokens
    Pz = rand(K); Pz /= sum(Pz) # P(z)
    Pd_z = rand(N, K); Pd_z /= Pd_z.sum(axis=0) # P(d|z)
    Pw_z = rand(M, K); Pw_z /= Pw_z.sum(axis=0) # P(w|z)
    pars = Pz, Pd_z, Pw_z   # pack parameters in a variable called pars
    return pars

In [146]:
n.shape

(772, 1665)

In [40]:
random_init_pars(K, n.shape)

(array([0.4001385, 0.5998615]),
 array([[0.11300014, 0.13423582],
        [0.15208975, 0.14145356],
        [0.11296352, 0.14415165],
        [0.10676998, 0.05131162],
        [0.14655433, 0.0403815 ],
        [0.02229351, 0.17183628],
        [0.11544259, 0.166549  ],
        [0.14183071, 0.10509881],
        [0.08905547, 0.04498175]]),
 array([[0.01186479, 0.08050633],
        [0.12389336, 0.03654783],
        [0.03002695, 0.18772072],
        [0.08509484, 0.13924188],
        [0.17923298, 0.00066867],
        [0.12707695, 0.13626144],
        [0.17833826, 0.14977082],
        [0.0883424 , 0.14072519],
        [0.03751731, 0.06700934],
        [0.09520133, 0.02886329],
        [0.0124025 , 0.02764159],
        [0.03100834, 0.0050429 ]]))

In [147]:
def likelihood(pars, docs2bow):
    Pz, Pd_z, Pw_z = pars   # unpack parameters
    L = 0
    # iterate over data in docs2bow and calculate prob of co-occur for them, based on pars
    for d, doc2bow in enumerate(docs2bow):
        for w, ndw in doc2bow:
            Pcocur = sum(Pz[:] * Pd_z[d,:] * Pw_z[w, :])    # P(d,w)
            # adding up all log-likelihood terms
            L += ndw * np.log(Pcocur)
    return L

In [148]:
likelihood(random_init_pars(K, n.shape), docs2bow)

-226363.2781683731

## EM

In [149]:
# Expectation step
def Estep(pars, docs2bow):  # no necessity to pass docs2bow (data) to Estep, but it'll help to decrease computations
    Pz, Pd_z, Pw_z = pars
    posters = np.zeros((len(Pz), len(Pd_z), len(Pw_z)))
    # posters could be an attribute and no need to reset to zeros because it's not accumulative
    # iterate through data and calculate posteriors just for seen pairs of (d, w)
    # so unseen posteriors left to be zero
    for z in range(len(Pz)):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                posters[z, d, w] = Pz[z] * Pd_z[d, z] * Pw_z[w, z]
    # normalization
    posters /= posters.sum(axis=0) + 1e-16  # a tiny number added just to avoid dividing by zero error for unseen (d, w)s
    return posters

In [150]:
posters = Estep(random_init_pars(K, n.shape), docs2bow)
posters

array([[[0.70265427, 0.34994321, 0.42926269, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.09259948, 0.01777886, 0.10245468, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0. 

In [151]:
posters.shape

(5, 772, 1665)

In [152]:
# is posters normalized?
posters.sum(axis=0)

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [153]:
# Maximization step
def Mstep(posters, docs2bow):
    # re-estimation of the parameters by posteriors calculated in E-step based on parameters
    K, N, M = posters.shape  # K, N, M could be in an attribute self.archit
    rePz, rePd_z, rePw_z = np.zeros(K), np.zeros((N, K)), np.zeros((M, K))
    # repars should reset to zeros in each M-step because they'd be calculated accumulatively
    # iterate over data and add up terms n(d,w) * poster(z|d,w) to associated repars
    for z in range(K):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                rePz[z] += ndw * posters[z, d, w]
                rePd_z[d, z] += ndw * posters[z, d, w]
                rePw_z[w, z] += ndw * posters[z, d, w]
    # normalization
    rePz /= sum(rePz)
    rePd_z /= rePd_z.sum(axis=0)
    rePw_z /= rePw_z.sum(axis=0)
    repars = rePz, rePd_z, rePw_z   # pack re-estimated parameters in repars and return it
    return repars

In [59]:
# just one EM-step
pars = random_init_pars(K, n.shape)
print(pars)
print(likelihood(pars, docs2bow))
# EM
posters = Estep(pars, docs2bow)
repars = Mstep(posters, docs2bow)
print(repars)
print(likelihood(repars, docs2bow))

(array([0.24922167, 0.75077833]), array([[0.09677253, 0.1602031 ],
       [0.00775021, 0.21583797],
       [0.00428237, 0.01857995],
       [0.09641279, 0.15206515],
       [0.24054326, 0.01789933],
       [0.06312533, 0.00745956],
       [0.09122164, 0.07598723],
       [0.16340418, 0.08003833],
       [0.2364877 , 0.2719294 ]]), array([[0.10680976, 0.03685408],
       [0.11841676, 0.0282922 ],
       [0.01953122, 0.13461726],
       [0.1143877 , 0.08770452],
       [0.07620412, 0.0837982 ],
       [0.07945055, 0.13575207],
       [0.04408571, 0.10852351],
       [0.10540922, 0.07066863],
       [0.00814478, 0.11543443],
       [0.1067055 , 0.10708165],
       [0.11714257, 0.04682082],
       [0.10371211, 0.04445263]]))
-141.13914367519305
(array([0.29152057, 0.70847943]), array([[0.10079388, 0.10454049],
       [0.00673905, 0.28925608],
       [0.04209003, 0.17736708],
       [0.07249001, 0.16485829],
       [0.27394979, 0.0332914 ],
       [0.08715156, 0.01281094],
       [0.0926756

In [60]:
# check whether parameters remain normalized
Pz, Pd_z, Pw_z = repars
print(Pz.sum(axis=0))
print(Pd_z.sum(axis=0))
print(Pw_z.sum(axis=0))

1.0
[1. 1.]
[1. 1.]


In [154]:
# Expectation Maximization steps
def EMsteps(runtimes, pars, docs2bow):
    print(pars)
    print(likelihood(pars, docs2bow))
    for runtime in range(runtimes):
        posters = Estep(pars, docs2bow)
        pars = Mstep(posters, docs2bow)
        print(likelihood(pars, docs2bow))
    print(pars)

In [62]:
# EM-step several times
pars = random_init_pars(K, n.shape)
EMsteps(20, pars, docs2bow)

(array([0.49919619, 0.50080381]), array([[0.00540957, 0.12289335],
       [0.22141974, 0.17191716],
       [0.01317461, 0.12508   ],
       [0.12972748, 0.14250206],
       [0.18448682, 0.01071087],
       [0.0221535 , 0.16361161],
       [0.10217125, 0.05755784],
       [0.15699478, 0.17376754],
       [0.16446224, 0.03195957]]), array([[0.01878284, 0.12819708],
       [0.08641681, 0.00456281],
       [0.14649302, 0.00887281],
       [0.08354953, 0.15345632],
       [0.1111021 , 0.13396   ],
       [0.16075068, 0.09740126],
       [0.02900145, 0.08714917],
       [0.0235575 , 0.02555719],
       [0.06534193, 0.11582632],
       [0.09665643, 0.08438132],
       [0.06355762, 0.130665  ],
       [0.1147901 , 0.02997071]]))
-141.17278431330504
-130.70270083840904
-127.63669856803766
-124.14865627778187
-122.03110881423996
-121.34445581430512
-121.06149326921965
-120.83685580360137
-120.59288899356928
-120.33518770727943
-120.1202055580319
-119.98218733077078
-119.90025916872264
-119.84419

In [63]:
# now you can judge the result. We'll do it formally later
token2id

{'human': 0,
 'interface': 1,
 'computer': 2,
 'survey': 3,
 'user': 4,
 'system': 5,
 'response': 6,
 'time': 7,
 'eps': 8,
 'trees': 9,
 'graph': 10,
 'minors': 11}

In [64]:
corpus

['Human machine interface for Lab ABC computer applications',
 'A survey of user opinion of computer system response time',
 'The EPS user interface management system',
 'System and human system engineering testing of EPS',
 'Relation of user-perceived response time to error measurement',
 'The generation of random, binary, unordered trees',
 'The intersection graph of paths in trees',
 'Graph minors IV: Width of trees and well-quasi-ordering',
 'Graph minors: A survey']

## TEM

In [155]:
# Tempered Expectation step with control parameter beta
def TEstep(beta, pars, docs2bow):
    Pz, Pd_z, Pw_z = pars
    posters = np.zeros((len(Pz), len(Pd_z), len(Pw_z)))
    for z in range(len(Pz)):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                posters[z, d, w] = (Pz[z] * Pd_z[d, z] * Pw_z[w, z])**beta  # beta
    posters /= posters.sum(axis=0) + 1e-16
    return posters

In [156]:
# Tempered Maximization step
def TMstep(posters, docs2bow):  # note beta has no role in TM-step. it played its role in TE-step
    K, N, M = posters.shape
    rePz, rePd_z, rePw_z = np.zeros(K), np.zeros((N, K)), np.zeros((M, K))
    for z in range(K):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                rePz[z] += ndw * posters[z, d, w]
                rePd_z[d, z] += ndw * posters[z, d, w]
                rePw_z[w, z] += ndw * posters[z, d, w]
    rePz /= sum(rePz)
    rePd_z /= rePd_z.sum(axis=0)
    rePw_z /= rePw_z.sum(axis=0)
    repars = rePz, rePd_z, rePw_z
    return repars

## Split data to train and held-out

In [157]:
len(docs2bow)

772

In [68]:
docs2bow

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (2, 1), (5, 1), (6, 1), (7, 1)],
 [(8, 1), (4, 1), (1, 1), (5, 1)],
 [(5, 2), (0, 1), (8, 1)],
 [(4, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(10, 1), (9, 1)],
 [(10, 1), (11, 1), (9, 1)],
 [(10, 1), (11, 1), (3, 1)]]

In [158]:
# iterate over the corpus and randomly erase words
# erased words will be writed in held-out corpus
# unerased words remain in corpus as training corpus
from numpy.random import randint

docs2bow_train, docs2bow_heldout = list(), list()
for doc2bow in docs2bow:
    doc2bow_train, doc2bow_heldout = list(), list()
    for w, ndw in doc2bow:
        ndw_train = randint(ndw+1)
        if ndw_train > 0:
            doc2bow_train += [(w, ndw_train)]
        if ndw - ndw_train > 0:
            doc2bow_heldout += [(w, ndw - ndw_train)]
    docs2bow_train += [(doc2bow_train)]
    docs2bow_heldout += [(doc2bow_heldout)]

In [159]:
len(docs2bow_heldout)

772

In [71]:
docs2bow_train

[[(0, 1)],
 [(4, 1), (5, 1), (6, 1)],
 [(8, 1), (4, 1), (5, 1)],
 [(5, 1), (0, 1), (8, 1)],
 [(4, 1), (6, 1), (7, 1)],
 [],
 [],
 [(10, 1), (11, 1)],
 [(11, 1)]]

In [160]:
len(docs2bow_train)

772

In [73]:
docs2bow_heldout

[[(1, 1), (2, 1)],
 [(3, 1), (2, 1), (7, 1)],
 [(1, 1)],
 [(5, 1)],
 [],
 [(9, 1)],
 [(10, 1), (9, 1)],
 [(9, 1)],
 [(10, 1), (3, 1)]]

## TEM for train and heldout

In [161]:
# modify likelihood to solve the problem of omitted words or docs
# in splitting, may some words be omitted entirely from training corpus
# therefore their condit probs P(w|z) remain zero in training procedure
# why? note that EM-steps works only with training corpus and obviously omitted words left unseen in training
# and also note that repars accumulate from zero for "seen" data, so repars for unseen data remain zero through M-step
# so in evaluating performance on held-out data by likelihood(pars, heldout), Pcocur(omitted) would be 0 (P(w|z) = 0)
# and it diverges log-likelihood!
# so for avoiding this problem, we ignore omitted words in log-likelihood calculations
# similar problem could happen for omitted docs
def likelihood(pars, docs2bow):
    Pz, Pd_z, Pw_z = pars
    L = 0
    for d, doc2bow in enumerate(docs2bow):
        for w, ndw in doc2bow:
            Pcocur = sum(Pz[:] * Pd_z[d,:] * Pw_z[w, :])
            # modification
            if Pcocur == 0: continue
            L += ndw * np.log(Pcocur)
    return L

In [186]:
# TEM-steps on training corpus and evaluating performance by likelihood on held-out corpus
def TEMsteps(beta_runtimes, first_beta, eta, docs2bow_train, docs2bow_heldout):
    beta = first_beta
    pars = random_init_pars(K, n.shape)
    for i in range(beta_runtimes):
        print(beta)
        # pars = random_init_pars(K, n.shape)
        # print(pars)
        new_likeli = likelihood(pars, docs2bow_heldout) # first likelihood in new beta
        likeli = -np.inf    # for assuring the entrance to while loop
        while round(new_likeli, 0) > round(likeli, 0):  # check if likelihood increased?
            likeli = new_likeli
            print(likeli)   # print increased likelihood (or first likelihood)
            # one TEM step
            prepars = pars  # save pars before executing TEM, for undoing pars if khodayi nakarde likelihood decreased
            posters = TEstep(beta, pars, docs2bow_train)    # TE-step
            pars = TMstep(posters, docs2bow_train)  # TM-step
            # print(pars)
            new_likeli = likelihood(pars, docs2bow_heldout) # calculating likelihood for re-estimated pars
        print(new_likeli)   # print decreased (inappropriate) likelihood
        pars = prepars  # undo pars
        beta *= eta # new beta
    return pars

In [187]:
# to find the approp beta, try TEM for various betas, on training and held-out corpus
TEMsteps(10, 1.00, 0.90, docs2bow_train, docs2bow_heldout)

1.0
-113658.16715716878
-97510.01244525202
-97598.50956036263
0.9
-97510.01244525202
-97548.26407456916
0.81
-97510.01244525202
-97508.38396533884
-97524.065901426
0.7290000000000001
-97508.38396533884
-97486.54552178112
-97474.19597415389
-97468.45984159262
-97468.32190710164
0.6561000000000001
-97468.45984159262
-97434.71053913893
-97408.80123733406
-97387.1075943421
-97367.43098512804
-97348.39256837049
-97329.14308710542
-97309.2045024375
-97288.3734782886
-97266.64959316835
-97244.17659450874
-97221.19336297746
-97197.99150785209
-97174.88183727597
-97152.16983590128
-97130.13633601705
-97109.02356522228
-97089.02748591639
-97070.28924866786
-97052.8743201667
-97036.74029448898
-97021.71835885331
-97007.5741548087
-96994.19017005584
-96981.61152147684
-96969.88944247583
-96959.02833295244
-96949.0162013715
-96939.8434775828
-96931.4960523202
-96923.93095644191
-96917.05907700672
-96910.75564310375
-96904.88423617401
-96899.30935527978
-96893.90175869946
-96888.55410935756
-96883.2

(array([0.20346026, 0.1938115 , 0.1948203 , 0.20442034, 0.20348759]),
 array([[6.52683974e-04, 2.40287547e-03, 1.15355680e-03, 6.89231677e-04,
         3.89221319e-03],
        [3.90595546e-03, 6.58848709e-04, 5.73917354e-05, 2.14121436e-04,
         1.28564506e-04],
        [1.19963469e-03, 1.16213427e-03, 4.65005421e-04, 5.06675418e-04,
         4.38129844e-04],
        ...,
        [0.00000000e+00, 0.00000000e+00, 6.43870457e-04, 0.00000000e+00,
         0.00000000e+00],
        [5.45433321e-04, 7.22620521e-04, 4.71783329e-03, 2.91101135e-03,
         5.71827133e-04],
        [1.11109713e-03, 5.55535376e-04, 9.39326303e-04, 7.16222286e-03,
         1.36158426e-03]]),
 array([[3.50174299e-04, 1.74246968e-04, 5.85907163e-05, 4.46679374e-05,
         6.15834880e-04],
        [7.47969026e-05, 1.38651755e-04, 9.00060734e-05, 1.60252232e-04,
         1.39533215e-03],
        [4.27967182e-05, 9.36223893e-04, 1.02561909e-04, 5.00486489e-04,
         3.13866580e-04],
        ...,
        [2.

## Final TEM by appropriate beta

In [188]:
# to obtain the learned pars, one TEM by approp beta, on the whole corpus
tem_learned_pars = TEMsteps(1, 0.70, 0.00, docs2bow, docs2bow)

0.7
-227081.4216625212
-207539.1340630666
-207403.43193742802
-207360.56011281666
-207335.16064082767
-207306.53298777374
-207267.46162899278
-207214.93243243944
-207147.36591002787
-207063.71634497808
-206963.19100878327
-206845.16362740836
-206709.13677923274
-206554.71677200808
-206381.5872417804
-206189.48660176725
-205978.26538825408
-205748.14022532417
-205500.1351581904
-205236.46505645325
-204960.5509038286
-204676.61453585039
-204389.11846657872
-204102.35702490743
-203820.27740882421
-203546.41090292393
-203283.8053208173
-203034.94040972428
-202801.65374461038
-202585.1034750867
-202385.75932810988
-202203.4087647685
-202037.21008903623
-201885.83075056376
-201747.66235012925
-201621.05639144254
-201504.50486553923
-201396.7189385756
-201296.63263999904
-201203.39235597505
-201116.33446533332
-201034.90993185088
-200958.57975304095
-200886.77489514338
-200818.95815829275
-200754.72599792253
-200693.8772431202
-200636.38822187073
-200582.28668545047
-200531.52767059903
-20048

In [104]:
# one EM (TEM by beta = 1), on the whole corpus
em_learned_pars = TEMsteps(1, 1.00, 0.00, docs2bow, docs2bow)

1.0
-134.95083987283653
-129.03858484983505
-126.8821700637002
-124.79179094753185
-122.6024058997259
-120.55752118757084
-119.30317783424297
-118.6376319462297
-118.23929456252168
-117.99123080907067
-117.86316109375669
-117.8129033307111
-117.79664605470663
-117.79180365670419
-117.79035348392607


In [99]:
# just for curiosity, let's calculate likelihood by ndw * log(ndw/N)
def likelihood(pars, docs2bow):
    Pz, Pd_z, Pw_z = pars
    L = 0
    N = sum(idf)    # N: total number of co-occurs, means total number of words seen in documents
    for d, doc2bow in enumerate(docs2bow):
        for w, ndw in doc2bow:
            # Pcocur = sum(Pz[:] * Pd_z[d,:] * Pw_z[w, :])
            # if Pcocur == 0: continue
            # L += ndw * np.log(Pcocur)
            L += ndw * np.log(ndw/N)
    return L

In [100]:
pars = 0, 0, 0
likelihood(pars, docs2bow)

-120.11369315764638

In [103]:
# return likelihood to its original definition
def likelihood(pars, docs2bow):
    Pz, Pd_z, Pw_z = pars
    L = 0
    for d, doc2bow in enumerate(docs2bow):
        for w, ndw in doc2bow:
            Pcocur = sum(Pz[:] * Pd_z[d,:] * Pw_z[w, :])
            # modification
            if Pcocur == 0: continue
            L += ndw * np.log(Pcocur)
    return L

## Looking at parameters

In [189]:
id2token = {tokenid: token for token, tokenid in token2id.items()}

In [108]:
id2token

{0: 'human',
 1: 'interface',
 2: 'computer',
 3: 'survey',
 4: 'user',
 5: 'system',
 6: 'response',
 7: 'time',
 8: 'eps',
 9: 'trees',
 10: 'graph',
 11: 'minors'}

In [176]:
def token2Pw_z(z, pars, first_probs=None):
    Pz, Pd_z, Pw_z = pars
    token2Pw_z = {id2token[tokenid]: prob for tokenid, prob in enumerate(Pw_z[:, z])}
    sorted_token2Pw_z = dict(sorted(token2Pw_z.items(), key=lambda item: -item[1])[:first_probs])
    return sorted_token2Pw_z

In [190]:
token2Pw_z(0, tem_learned_pars, first_probs=15)

{'vaccines': 0.03423843433582849,
 '19': 0.033328790264425395,
 'covid': 0.03298085730344681,
 'is': 0.019284414183332058,
 'in': 0.019006113054894944,
 'doses': 0.013304239431646974,
 'this': 0.013281095891384763,
 'countries': 0.0127042412587594,
 'are': 0.011661124362352237,
 'unicef': 0.011118740241183851,
 'vaccine': 0.010578587176592914,
 '@unicefchief': 0.010441576009800124,
 'today': 0.009999726848945711,
 'india': 0.009647629734884719,
 'have': 0.009173755255073893}

In [191]:
token2Pw_z(1, tem_learned_pars, first_probs=15)

{'is': 0.03729376870162209,
 'water': 0.032300314684034714,
 'we': 0.026889746105528554,
 'in': 0.0218063062386254,
 'climate': 0.01989892328255509,
 'are': 0.01803323548635282,
 'from': 0.012976710367431277,
 'action': 0.012401446420656331,
 'now': 0.012135623707899713,
 'with': 0.012050142769226111,
 'safe': 0.009617434558251684,
 'future': 0.009350197035551193,
 'this': 0.009222506883361915,
 'change': 0.00886858451884581,
 'crisis': 0.008832030971419784}

In [194]:
token2Pw_z(2, tem_learned_pars, first_probs=15)

{'in': 0.029473994228659463,
 'covid': 0.019288054109475844,
 '19': 0.018958365176421736,
 'are': 0.017679593175088177,
 'children': 0.017143285042069967,
 'their': 0.016652776833343586,
 'is': 0.01628870429919858,
 'education': 0.014888863335156727,
 'school': 0.013763948711660106,
 'we': 0.012611173613753314,
 'must': 0.011191456555053195,
 'on': 0.01021240013947872,
 'access': 0.009387882137598712,
 'they': 0.008999724605934914,
 'learning': 0.008929128615130224}

In [192]:
token2Pw_z(3, tem_learned_pars, first_probs=15)

{'in': 0.06428183227504712,
 'children': 0.05038918208519468,
 'on': 0.017994741184701426,
 'is': 0.01692342683468041,
 'violence': 0.013826703729379013,
 'have': 0.012450435205970476,
 'families': 0.01102249269860199,
 '@unicefchief': 0.01049023817899592,
 'killed': 0.010449559589558215,
 'by': 0.009964800683959005,
 '10': 0.00895961753590706,
 '@unicefpalestine': 0.008910493649853355,
 'at': 0.008729763209737678,
 'conflict': 0.008325010226054556,
 'been': 0.008018373772816957}

In [197]:
token2Pw_z(4, tem_learned_pars, first_probs=15)

{'you': 0.02179684755242624,
 '19': 0.019116289047192248,
 'covid': 0.019110793537997266,
 'your': 0.019103158093027994,
 'our': 0.018807919834677823,
 'how': 0.016013071933924292,
 'health': 0.015282297567660374,
 'on': 0.014990690853953634,
 'is': 0.013670774198825644,
 'as': 0.011646271469317439,
 'out': 0.010977041131326701,
 'can': 0.010956114825726674,
 'in': 0.009524654773668435,
 'mental': 0.009102908423962655,
 'this': 0.008827114777684006}